In [210]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import pearsonr

## Read in and clean TikTok data

In [213]:
df = pd.read_csv("sportswear.csv")

# Add a column for diggs-to-plays ratio
df['diggCount/playCount'] = df['diggCount'] / df['playCount']

# Sort by time
df = df.sort_values(by='createTimeISO')

# Get date in y-m-d format from `df`
df['createTimeISO'] = pd.to_datetime(df['createTimeISO'])
df['date'] = df['createTimeISO'].dt.strftime('%Y-%m-%d')

# Create `cleaned_df`, keeping only the following columns of `df`
cleaned_df = df[['date', 'diggCount', 'playCount', 'diggCount/playCount']]

# Create `aggregated_df`, aggregating metadata for videos made on the same day.
# Ex:
# date                  diggCount
# 2019-11-01            200
# 2019-11-01            150
# will become:
# date                  diggCount
# 2019-11-01            350
aggregated_df_sport = cleaned_df.groupby('date', as_index = False)[['diggCount', 'playCount', 'diggCount/playCount']].sum()

aggregated_df_sport['date'] = pd.to_datetime(aggregated_df_sport['date']).dt.date

#aggregated_df_sport

In [215]:
df = pd.read_csv("athleticwear.csv")

# Add a column for diggs-to-plays ratio
df['diggCount/playCount'] = df['diggCount'] / df['playCount']

# Sort by time
df = df.sort_values(by='createTimeISO')

# Get date in y-m-d format from `df`
df['createTimeISO'] = pd.to_datetime(df['createTimeISO'])
df['date'] = df['createTimeISO'].dt.strftime('%Y-%m-%d')

# Create `cleaned_df`, keeping only the following columns of `df`
cleaned_df = df[['date', 'diggCount', 'playCount', 'diggCount/playCount']]

# Create `aggregated_df`, aggregating metadata for videos made on the same day.
# Ex:
# date                  diggCount
# 2019-11-01            200
# 2019-11-01            150
# will become:
# date                  diggCount
# 2019-11-01            350
aggregated_df_athwear = cleaned_df.groupby('date', as_index = False)[['diggCount', 'playCount', 'diggCount/playCount']].sum()

aggregated_df_athwear['date'] = pd.to_datetime(aggregated_df_athwear['date']).dt.date

#aggregated_df_athwear

In [217]:
df = pd.read_csv("athleisure.csv")

# Add a column for diggs-to-plays ratio
df['diggCount/playCount'] = df['diggCount'] / df['playCount']

# Sort by time
df = df.sort_values(by='createTimeISO')

# Get date in y-m-d format from `df`
df['createTimeISO'] = pd.to_datetime(df['createTimeISO'])
df['date'] = df['createTimeISO'].dt.strftime('%Y-%m-%d')

# Create `cleaned_df`, keeping only the following columns of `df`
cleaned_df = df[['date', 'diggCount', 'playCount', 'diggCount/playCount']]

# Create `aggregated_df`, aggregating metadata for videos made on the same day.
# Ex:
# date                  diggCount
# 2019-11-01            200
# 2019-11-01            150
# will become:
# date                  diggCount
# 2019-11-01            350
aggregated_df_athlei = cleaned_df.groupby('date', as_index = False)[['diggCount', 'playCount', 'diggCount/playCount']].sum()

aggregated_df_athlei['date'] = pd.to_datetime(aggregated_df_athlei['date']).dt.date

#aggregated_df_athlei

## Get and clean stock data

In [199]:
# Get prices of specified stock for each day
ticker = "ADDYY"
data = yf.Ticker(ticker)
prices_ad = data.history(start = '2019-10-01', end = '2024-11-16') # can change time interval; prices is a dataframe

# Reset indices of `prices`, dropping old indices. 
prices_ad.reset_index(inplace=True)

prices_ad['Date'] = pd.to_datetime(prices_ad['Date']).dt.date

#prices_ad

In [201]:
# Get prices of specified stock for each day
ticker = "NKE"
data = yf.Ticker(ticker)
prices_nk = data.history(start = '2019-10-01', end = '2024-11-16') # can change time interval; prices is a dataframe

# Reset indices of `prices`, dropping old indices. 
prices_nk.reset_index(inplace=True)

prices_nk['Date'] = pd.to_datetime(prices_nk['Date']).dt.date

#prices_nk

In [203]:
# Get prices of specified stock for each day
ticker = "LULU"
data = yf.Ticker(ticker)
prices_lu = data.history(start = '2019-10-01', end = '2024-11-16') # can change time interval; prices is a dataframe

# Reset indices of `prices`, dropping old indices. 
prices_lu.reset_index(inplace=True)

prices_lu['Date'] = pd.to_datetime(prices_lu['Date']).dt.date

#prices_lu

## Merging data and calculating correlation

In [206]:
hashtags_list = [aggregated_df_sport, aggregated_df_athwear, aggregated_df_athlei]
prices_list = [prices_ad, prices_nk, prices_lu]

In [208]:
# print(cleaned_df.head(5))
# print(prices.head(5))

# print("cleaned_df dtypes: \n" + cleaned_df.dtypes)
# print("prices dtypes: \n" + prices.dtypes)

for aggregated_df in hashtags_list:
    for prices in prices_list:

        # Merge `cleaned_df` and `prices`, based on dates (use inner merge - only include data that is common to both dataframes)
        merged_df = pd.merge(
            aggregated_df,
            prices[['Date', 'Open', 'Close']],
            left_on = aggregated_df['date'],
            right_on = prices['Date'],
            # left_on = pd.to_datetime(aggregated_df['date']).dt.date,
            # right_on = pd.to_datetime(prices['Date']).dt.date,
            how = 'inner'
            )
        
        #calculate daily change in ticker price
        merged_df['ChangePriceDay'] = merged_df['Open'] - merged_df['Close']
        
        #print(merged_df)
        
        # Only keep relevant columns (use closing prices of stock for now)
        merged_df = merged_df[['date', 'diggCount', 'playCount', 'diggCount/playCount', 'ChangePriceDay']]
        
        # Calculate percentage change in diggCount/playCount 
        merged_df['change in d/p'] = merged_df['diggCount/playCount'].pct_change()
        
        # Calculate percentage change per time
        merged_df['deltaTime'] = merged_df['date'].diff().dt.days
        
        merged_df['change'] = merged_df['change in d/p'] / merged_df['deltaTime']
        
        merged_df = merged_df[['date', 'change', 'ChangePriceDay', 'diggCount/playCount']]
        
        #print(merged_df.dtypes)
        
        merged_df
        
        corr, p_value = pearsonr(merged_df['diggCount/playCount'], merged_df['ChangePriceDay'])
        print("Correlation:", corr)
        print("P-value:", p_value)

Correlation: -0.15998004193691073
P-value: 0.2220804425820424
Correlation: -0.0763301128571158
P-value: 0.5621441533494591
Correlation: -0.04402933205751826
P-value: 0.7383512861830089
Correlation: 0.04062450435346725
P-value: 0.7727207827189371
Correlation: 0.14356871437286609
P-value: 0.3050810164619881
Correlation: 0.04546395825109847
P-value: 0.746500267529089
Correlation: 0.022571336264228074
P-value: 0.7942248838925293
Correlation: -0.09471601542989205
P-value: 0.2727106851309132
Correlation: -0.008431076339954248
P-value: 0.9223955745811074


## Strategy (daily)

In [123]:
class Strategy:
    def __init__(self, data, balance = 1000000):
        self.data = data
        self.balance = balance
        self.shares = 0
        self.transactions = 0

    # Helper function that gets bid/ask price
    def get_price(self, date):
        row = self.data[self.data['date'] == date]
        if not row.empty:
            return row['Close'].values[0]
        else:
            return None
            

    # Buy as many shares as possible with remaining balance
    def buy(self, date):
        if self.balance > 0:
            bid = self.get_price(date)
            self.shares = self.balance / bid 
            print(f"Bought {self.shares} shares at {bid} \n Balance: {self.balance}    Shares: {self.shares}")
            self.balance = 0
            self.transactions += 1
        else:
            print("Balance too low to buy") 

    # Sell as many shares as possible given ask pricce
    def sell(self, date):
        if self.shares > 0:
            ask = self.get_price(date)
            self.balance += self.shares * ask
            print(f"Sold {self.shares} shares at {ask} \n Balance: {self.balance}    Shares: {self.shares}")
            self.shares = 0
            self.transactions += 1
        else:
            print("No shares to sell") 
            
    # Compare change 
    def run(self):
        print(f"Starting balance: {self.balance}")
        print(f"Starting shares: {self.shares}")
        print(f"Number of transactions: {self.transactions} \n")
        
        # Iterate through each row in `data` and print the date, followed by the action taken (buy/sell) and the results
        for i in range(1, len(self.data)): 
            # Print date
            print(self.data['date'].iloc[i], end = ": ")

            # If the change in diggCount/playCount is larger than 6 (arbitrarily set)
            if self.data['change'].iloc[i] > 2.0:
                self.buy(self.data['date'].iloc[i])
            elif self.data['change'].iloc[i] < 0.0:
                self.sell(self.data['date'].iloc[i])
            else:
                continue
            
        print(f"\nEnding balance: {self.balance}")
        print(f"Ending shares: {self.shares}")
        print(f"Number of transactions: {self.transactions}")

## Run strategy

In [53]:
strategy = Strategy(merged_df, balance = 1000000)
strategy.run()

Starting balance: 1000000
Starting shares: 0
Number of transactions: 0 

2019-11-20: No shares to sell
2019-11-21: Bought 4613.822896814219 shares at 216.74000549316406 
 Balance: 1000000    Shares: 4613.822896814219
2019-12-31: Sold 4613.822896814219 shares at 231.6699981689453 
 Balance: 1068884.342056788    Shares: 4613.822896814219
2020-01-02: Bought 4579.232072837 shares at 233.4199981689453 
 Balance: 1068884.342056788    Shares: 4579.232072837
2020-01-03: 2020-01-06: Sold 4579.232072837 shares at 235.42999267578125 
 Balance: 1078088.5733687175    Shares: 4579.232072837
2020-01-14: No shares to sell
2020-01-21: 2020-01-22: Bought 4412.9699545697995 shares at 244.3000030517578 
 Balance: 1078088.5733687175    Shares: 4412.9699545697995
2020-01-23: Sold 4412.9699545697995 shares at 243.05999755859375 
 Balance: 1072616.466383883    Shares: 4412.9699545697995
2020-01-24: No shares to sell
2021-03-02: No shares to sell
2021-08-04: 2021-09-09: No shares to sell
2022-01-21: No shares 

## Notes
problems: scraper is weird, percent change calculation is off bc we don't have data every day (jumps multiple days), etc
modifications: set different thresholds for when to buy/sell, set diff amounts for buying/selling, transaction fee (?), buying non-whole shares